# **<u>3. 다중선형회귀(Multivariable Linear Regression) </u>**
* $x$가 1개인 선형회귀를 단순 선형회귀라고 함. 
* 이번 챕터에서는 다수의 $x$로부터 $y$를 예측하는 **다중선형회귀**에 대해서 학습

## **1.데이터에 대한 이해(Data Definition)**
* 3개의 퀴즈 점수로부터 최종 점수를 예측하는 모델 
  * $x$의 개수가 3개인 훈련데이터
  * $H(x)=w_1x_1+w_2x_2+w_3x_3+b$

## **2.파이토치로 구현하기**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

In [2]:
torch.manual_seed(1) #랜덤 시드 고정

In [3]:
# 훈련데이터 선언
x1_train=torch.FloatTensor([[73],[93],[89],[96],[73]])
x2_train=torch.FloatTensor([[80],[88],[91],[98],[66]])
x3_train=torch.FloatTensor([[75],[93],[90],[100],[70]])

y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

In [4]:
# 가중치 w와 편향 b 선언
w1=torch.zeros(1,requires_grad=True)
w2=torch.zeros(1,requires_grad=True)
w3=torch.zeros(1,requires_grad=True)

b=torch.zeros(1,requires_grad=True)

In [7]:
#optimizer 설정
optimizer=optim.SGD([w1,w2,w3,b],lr=1e-5)
nb_epochs=1000


for epoch in range(nb_epochs+1):
    #H(x)
    hypothesis=x1_train*w1+x2_train*w2+x3_train*w3+b
    
    #cost
    cost=torch.mean((y_train-hypothesis)**2) 

    #cost로 H(x) 개선
    optimizer.zero_grad() # 파라메터의 gradient 0으로 초기화
    cost.backward() #gradient 계산
    optimizer.step() #update parameter 

    #100번마다 로그 출력
    if epoch % 100==0:
        print('Epoch {:4d}/{} w1:{:.3f} w2:{:.3f} w3:{:.3f} b{:.3f} Cost:{:.6f}'.format(epoch,nb_epochs,w1.item(),w2.item(),w3.item(),b.item(),cost.item()))

Epoch    0/1000 w1:0.551 w2:0.550 w3:0.557 b0.006 Cost:2915.712158
Epoch  100/1000 w1:0.674 w2:0.661 w3:0.676 b0.008 Cost:1.562279
Epoch  200/1000 w1:0.679 w2:0.655 w3:0.677 b0.008 Cost:1.496321
Epoch  300/1000 w1:0.684 w2:0.649 w3:0.677 b0.008 Cost:1.433805
Epoch  400/1000 w1:0.690 w2:0.643 w3:0.678 b0.008 Cost:1.374573
Epoch  500/1000 w1:0.695 w2:0.638 w3:0.678 b0.009 Cost:1.318419
Epoch  600/1000 w1:0.699 w2:0.632 w3:0.679 b0.009 Cost:1.265176
Epoch  700/1000 w1:0.704 w2:0.627 w3:0.679 b0.009 Cost:1.214724
Epoch  800/1000 w1:0.709 w2:0.622 w3:0.679 b0.009 Cost:1.166892
Epoch  900/1000 w1:0.714 w2:0.617 w3:0.680 b0.009 Cost:1.121545
Epoch 1000/1000 w1:0.718 w2:0.612 w3:0.680 b0.009 Cost:1.078541


## **3. 벡터와 행렬 연산으로 바꾸기**
* $x$의 개수가 1000개라고 가정하면 x_train1~ x_train1000, w1~w1000까지 총 2000개의 변수를 선언해야함. 
* $x$와 $w$ 변수 선언을 효율적으로 처리하려면 어떻게 해야하는가. 
* **행렬 곱셈 연산(또는 벡터의 내적)** 사용 
### **(1) 벡터연산으로 이해하기**
* #